> **Установка необходимых библиотек**

Эти команды устанавливают программы, которые помогут нам анализировать и генерировать текст, а также хранить важные данные в безопасности.

In [ ]:
!pip install openai TextBlob python-dotenv

**Т.к. TextBlob некорректно работает с русским языком нужно установить переводчик**

In [ ]:
!pip install translate

> **Этот код подключает ваш Google Диск к рабочей среде Google Colab.**

После выполнения этих команд у вас появится доступ к файлам на вашем Google Диске прямо из Google Colab. Это нужно для того чтобы подтянуть ваш OpenAI API Key. Что-бы все работало коректно добвьте .env-файл(пример OPENAI_API_KEY ="Ваш API ключ") на ваш диск


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


**Этот код импротирует, загружает .env файл**


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from textblob import TextBlob
from translate import Translator

load_dotenv('/content/drive/MyDrive/.env')

client = OpenAI(api_key = os.environ.get('OPENAI_API_KEY'))

**Это основной код который анализирует текст пользователя, генерирует ответ в зависимости от того позитивный текст или негативный**

т.к. TextBlob некорректно работает с русским языком приходится переводить текст на английский  

**ТЕКСТ МОЖЕТ БЫТЬ НЕККОРЕТНЫМ Т.К. ПРОШЕЛ ЧЕРЕЗ ПЕРЕВОДЧИК**

In [61]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return "positive"
    else:
        return "negative"

def generate_response(prompt, character):
    if character == 'Batman':
        persona = "Respond as Batman from the DC Comics, using a serious and heroic tone."
    elif character == 'Joker':
        persona = "Respond as Joker from the DC Comics, using a chaotic and mischievous tone."
    chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": persona},
        { "role": "user", "content": prompt,}
    ],
    model="gpt-3.5-turbo",)
    return chat_completion.choices[0].message.content

def chatbot(questions):
    if questions == None:
        history = []
        translator_ru_to_en = Translator(from_lang='ru', to_lang='en')
        translator_en_to_ru = Translator(from_lang='en', to_lang='ru')
        while True:
            user_input = translator_ru_to_en.translate(input('Ваше предложение: '))
            if user_input.lower() == "serving":
                print("Конец чата")
                break
            if len(history) < 1:
                sentiment = analyze_sentiment(user_input)
                character = 'Batman' if sentiment == 'positive' else 'Joker'

            response = generate_response(user_input, character)
            print(translator_en_to_ru.translate(f"{character}: {response}"))

            history.append((user_input, response))

            if character == 'joker':
                next_question_prompt = "Ask question like Joker"
            else:
                next_question_prompt = "Ask question like Batman"

            next_question = generate_response(next_question_prompt, character)
            print(translator_en_to_ru.translate(f"{character}: {next_question}"))

            history.append((None, next_question))

            print("\nИстория чата:")
            print('Для прекращения чата напишите "Выход"')
            for i, (user_msg, bot_msg) in enumerate(history):
                if user_msg:
                    print(f"{i+1}. Вы: {translator_en_to_ru.translate(user_msg)}")
                if bot_msg:
                    print(f"{i+1}. {character}: {translator_en_to_ru.translate(bot_msg)}")
    else:
        for i, test_text in enumerate(questions):
            sentiment = analyze_sentiment(test_text)
            if sentiment == 'positive':
                character = "Batman"
            else:
                character = "Joker"
            response = generate_response(test_text, character)
            print(f'Test№{i+1}')
            print(f'Text: {test_text}')
            print(f'Answer: {character}: {response}')

**Запуск бота**

In [ ]:
chatbot(questions=None)

**Тесты**

Я их сразу на английском написал)

In [ ]:
test_queries = [
    "We all know Batman's always trying to do the right thing, even with everything he's been through.",
    "That Joker guy has this crazy charm and knows how to get into people's heads.",
    "Whenever Batman and the Joker face off, it feels like the whole city is on edge.",
    "Batman has all these cool gadgets and resources to tackle crime.",
    "The Joker will do just about anything to get what he wants, which makes him so unpredictable.",
    "Every time the Joker shows up, it seems like Gotham ends up in chaos.",
    "Sometimes it feels like Batman gets so fixated on his mission that he forgets other important stuff.",
    "The Joker has this way of twisting people’s fears to get what he wants.",
    "It’s like the Joker knows just how to push Batman’s buttons and make him lose it.",
    "No matter how hard Batman tries, it feels like the Joker always manages to slip away without getting caught."
]

chatbot(test_queries)